In [1]:
import re
from wordcloud import WordCloud
import pandas as pd
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from
from flask import Flask, jsonify
import sqlite3
import demoji
import emoji
from unidecode import unidecode
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter
import pickle
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# PREPARE MEMBUAT API

In [2]:
#fungsi untuk menghilangkan karakter
def clean_karakter(texts):
    tanpa_hex = re.sub(r'\\x..', ' ', texts)
    tanpa_backslash = re.sub(r'\\', ' ', tanpa_hex)
    tanpa_newline = re.sub(r'\n', ' ', tanpa_backslash)
    non_ascii = re.sub(r'[^\x00-\x7F]+', ' ', tanpa_newline) 
    non_url = re.sub(r'https?://\S+|www\.\S+', ' ', non_ascii) 
    non_whitespace = re.sub(r'\s+', ' ', non_url) 
    only_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', non_whitespace) 
    lower_text = re.sub(r'[A-Z]', lambda x: x.group(0).lower(), only_text) 
    
    tanpa_kata_user = re.sub(r'\buser\b', ' ', lower_text) 
    tanpa_kata_RT = re.sub(r'\brt\b', ' ', tanpa_kata_user) 
    cleaned_text_new = re.sub(r'\s+', ' ', tanpa_kata_RT).strip() 
    return cleaned_text_new

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_features = 100000
tokenizer = Tokenizer (num_words=max_features, split = ' ', lower = True)
sentiment = ['negative', 'neutral','positive']

In [4]:
file = open('platinum/resources_of_lstm/x_pad_sequences.pickle', 'rb')
feature_file_from_lstm = pickle.load(file)
file.close()

In [5]:
model_file_from_lstm = load_model ('platinum/model_of_lstm/model.h5')

In [6]:
import pickle

# Memuat kembali model dari berkas
with open("platinum/model_of_nn/model_mlp_classifier.pkl", "rb") as file:
    loaded_model = pickle.load(file)

C:\Users\akung\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\akung\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
import pickle

# Buka file untuk dibaca dalam mode biner ("rb")
with open("platinum/feature.p", "rb") as file:
    # Mengembalikan objek yang disimpan menggunakan pickle
    count_vect = pickle.load(file)

#  MEMBUAT API

In [8]:
app = Flask(__name__)

app.json_encoder = LazyJSONEncoder
swagger_template = {
    "info": {
        "title":  "API Documentation for Data Processing and Modeling",
        "version": "1.0.0",
        "description": "Dokumentasi API untuk Data Processing dan Modeling"
    },
    "host": "127.0.0.1:5000"
}

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'api',
            "route": '/api.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/api/"
}


swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)



@swag_from("C://Users/akung/api/neural_network.yml", methods=['POST'])
@app.route('/neural-network', methods=['POST'])
def text_processing_nn():
    original_text = request.form.get('text')
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', original_text)
    cleaned_text = re.sub(' +', ' ', cleaned_text).strip()
    # Mengubah teks menjadi vektor fitur
    text = count_vect.transform([cleaned_text])
    # Menggunakan model untuk memprediksi sentimen teks
    result = loaded_model.predict(text)[0]
    json_response = {
        'status_code': 200,
        'description': "Hasil Sentimen analisis menggunakan Neural Network",
        'text': cleaned_text,
        'sentiment': result
    }

    response_data = jsonify(json_response)
    return response_data


@swag_from("C://Users/akung/api/lstm.yml", methods=['POST'])
@app.route('/lstm', methods=['POST'])
def text_processing_lstm():
    original_text = request.form.get('text')
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', original_text)
    cleaned_text = re.sub(' +', ' ', cleaned_text).strip()
    feature = tokenizer.texts_to_sequences(cleaned_text)
    feature = pad_sequences(feature, maxlen=feature_file_from_lstm.shape[1])
    prediction = model_file_from_lstm.predict(feature)
    get_sentiment = sentiment[np.argmax(prediction[0])]
    json_response = {
        'status_code': 200,
        'description': "Hasil Sentimen analisis menggunakan LSTM",
        'data':{
            'text': original_text,
            'sentiment': get_sentiment
        }
    }

    response_data = jsonify(json_response)
    return response_data

#input file
@swag_from("C://Users/akung/api/neural_network_file.yml", methods=['POST'])
@app.route('/neural_networkfile', methods=['POST'])
def file_nn():
    sentiment_result = []
    file = request.files.getlist('file')[0]
    df_file = pd.read_csv(file,encoding='latin-1')
    df_file = df_file.drop_duplicates()
    for text in df_file.iloc[:, 0]:
        if text.strip():  
            cleaned_text = clean_karakter(text)
            text_count = count_vect.transform([cleaned_text])
            # Menggunakan model untuk memprediksi sentimen teks
            result_text = loaded_model.predict(text_count)[0]
            sentiment_result.append({
                'original teks': text,
                'clean teks': cleaned_text,
                'sentiment': result_text
            })

    json_response = {
        'status_code': 200,
        'description': "Hasil Sentimen analisis menggunakan Neural Network",
        'data': sentiment_result
    }
    response_data = jsonify(json_response)
    return response_data



#input file
@swag_from("C://Users/akung/api/lstm_file.yml", methods=['POST'])
@app.route('/lstm_file', methods=['POST'])
def file_lstm():
    sentiment_result = []
    file = request.files.getlist('file')[0]
    df_file = pd.read_csv(file,encoding='latin-1')
    df_file = df_file.drop_duplicates()

    
    for text in df_file.iloc[:, 0]:
        if text.strip(): 
            cleaned_text = clean_karakter(text)
            text_count = count_vect.transform([cleaned_text])
            result_text = loaded_model.predict(text_count)[0]
            sentiment_result.append({
                'original teks': text,
                'clean teks': cleaned_text,
                'sentiment': result_text
            })

    json_response = {
        'status_code': 200,
        'description': "Hasil Sentimen analisis menggunakan LSTM",
        'data': sentiment_result
    }
    response_data = jsonify(json_response)
    return response_data

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Mar/2024 02:17:11] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [02/Mar/2024 02:17:14] "GET /api/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2024 02:17:14] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Mar/2024 02:17:14] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [02/Mar/2024 02:17:14] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Mar/2024 02:17:14] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Mar/2024 02:17:14] "GET /api.json HTTP/1.1" 200 -


1/1 [==============================] - 0s 450ms/step


127.0.0.1 - - [02/Mar/2024 02:17:23] "POST /lstm HTTP/1.1" 200 -


1/1 [==============================] - 0s 35ms/step


127.0.0.1 - - [02/Mar/2024 02:17:49] "POST /lstm HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2024 02:18:14] "POST /lstm_file HTTP/1.1" 200 -
